In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import librosa
import math

In [2]:
s, sr_s=librosa.load('data/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr_x=librosa.load('data/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
print(S.shape)
print(X.shape)
S_abs = np.abs(S)
X_abs = np.abs(X)

(513, 2459)
(513, 2459)


In [23]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 50

x = tf.placeholder(tf.float32, [None, 513])
y_ = tf.placeholder(tf.float32, [None, 513])

l1 = tf.layers.dense(x, units=200,  activation=tf.nn.relu)
l2 = tf.layers.dense(l1, units=200,  activation=tf.nn.relu)
l3 = tf.layers.dense(l2, units=200,  activation=tf.nn.relu)
y = tf.layers.dense(l3, units=513, activation=tf.nn.relu)

# loss = tf.reduce_sum(tf.squared_difference(y , y_))
# loss = -tf.reduce_sum(y_*tf.log(y + 1e-10))
loss = tf.losses.mean_pairwise_squared_error(labels=y_, predictions=y)
train_step = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

sess=tf.Session()
tf.global_variables_initializer().run(session=sess)

train_data = tf.data.Dataset.from_tensor_slices(tf.constant(X_abs.T[:]))
label_data = tf.data.Dataset.from_tensor_slices(tf.constant(S_abs.T[:]))
zip_data = tf.data.Dataset.zip((train_data, label_data))

batch_data = zip_data.batch(BATCH_SIZE)
batch_data = batch_data.repeat()
iterator = batch_data.make_one_shot_iterator()
next_batch = iterator.get_next()


In [24]:

TRAIN_STEPS = 10000
for i in range(TRAIN_STEPS+1):
    batch = sess.run(next_batch)
    _, loss_value = sess.run((train_step, loss), feed_dict={x: batch[0], y_: batch[1]})
    if i% 1000 == 0:
#         loss_log.append(loss_value) 
        print('Training Step:' + str(i) + '  Loss =  ' + str(loss_value))



Training Step:0  Loss =  9.016011
Training Step:1000  Loss =  5.661606
Training Step:2000  Loss =  4.5196514
Training Step:3000  Loss =  3.8472893
Training Step:4000  Loss =  3.424275
Training Step:5000  Loss =  3.1361442
Training Step:6000  Loss =  2.8020625
Training Step:7000  Loss =  2.6297038
Training Step:8000  Loss =  2.4808888
Training Step:9000  Loss =  2.3541903
Training Step:10000  Loss =  2.2479632


In [28]:
# # Test output X
# S_test_abs = sess.run(y, feed_dict={x: X_abs.T}).T
# ratio = X / X_abs
# Sh = np.multiply(ratio, S_test_abs)
# librosa.output.write_wav('recover_s1o.wav', librosa.istft(Sh, hop_length=512), sr_x)
def denoise_sound(input_file_name, output_file_name):
    sn, sr=librosa.load(input_file_name, sr=None)
    testX=librosa.stft(sn, n_fft=1024, hop_length=512)
    testX_abs = np.abs(testX)
    S_test_abs = sess.run(y, feed_dict={x: testX_abs.T}).T
    ratio = (testX / testX_abs)
    Sh = np.multiply(ratio, S_test_abs)
    librosa.output.write_wav(output_file_name, librosa.istft(Sh, hop_length=512), sr)

In [29]:
# Recover sound track
denoise_sound('data/test_x_01.wav', 'recover_01.wav')
denoise_sound('data/test_x_02.wav', 'recover_02.wav')
